In [69]:
import sys
import time
import os
import subprocess
import math
import matplotlib.pyplot as plt
import numpy as np
from astropy.table import Table, Column 
from scipy.stats import linregress
from scipy import interpolate
from scipy import polyval, polyfit
from scipy.optimize import curve_fit
from scipy import odr
import pylab as py
from matplotlib import gridspec
import sklearn.datasets as ds
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
import corner
import emcee
import scipy.optimize as op
from scipy.linalg import cholesky, inv,det
from scipy.optimize import minimize
import random
from astropy.table import Table, Column
import astropy.stats as st

from calibTools import *

In [70]:
table   = np.genfromtxt('corrected_mags_linewidth_all.csv' , delimiter='|', 
                        filling_values=-1, names=True, dtype=None, encoding=None)


Au_e  = getReddening_err(table, band1='u', band2='w2')
Ag_e  = getReddening_err(table, band1='g', band2='w2')
Ar_e  = getReddening_err(table, band1='r', band2='w2')
Ai_e  = getReddening_err(table, band1='i', band2='w2')
Az_e  = getReddening_err(table, band1='z', band2='w2')
Aw1_e  = getReddening_err(table, band1='w1', band2='w2')
Aw2_e  = getReddening_err(table, band1='w2', band2='w2')


catal   = np.genfromtxt('ESN_HI_catal_all.csv' , delimiter=',', 
                        filling_values=-1, names=True, dtype=None, encoding=None)
pgc_catal   = catal['pgc']
R50u_catal  = catal['R50_u']
R50g_catal  = catal['R50_g']
R50r_catal  = catal['R50_r']
R50i_catal  = catal['R50_i']
R50z_catal  = catal['R50_z']
R50w1_catal = catal['R50_w1']
R50w2_catal = catal['R50_w2']
Sba_catal   = catal['Sba']
Wba_catal   = catal['Wba']


pgc    = table['pgc']
Name   = table['Name']
inc    = table['inc']
inc_e  = table['inc_e']
Wmx    = table['Wmx']
eWmx   = table['eWmx']
logWimx = table['logWimx']
logWimx_e  = table['logWimx_e']
m21    = table['m21']
m21_e  = table['m21_e']
uu     = table['u']
gg     = table['g']
rr     = table['r']
ii     = table['i']
zz     = table['z']
w1     = table['w1']
w2     = table['w2']


u_     = table['u_']
g_     = table['g_']
r_     = table['r_']
i_     = table['i_']
z_     = table['z_']
w1_    = table['w1_']
w2_    = table['w2_']

N = len(pgc)
R50u  = np.zeros(N)
R50g  = np.zeros(N)
R50r  = np.zeros(N)
R50i  = np.zeros(N)
R50z  = np.zeros(N)
R50w1 = np.zeros(N)
R50w2 = np.zeros(N)
Sba   = np.zeros(N)
Wba   = np.zeros(N)

u_s   = table['u_sss']
g_s   = table['g_sss']
r_s   = table['r_sss']
i_s   = table['i_sss']
z_s   = table['z_sss']
w1_s  = table['w1_sss']
w2_s  = table['w2_sss']


for i, idd in enumerate(pgc):
    if idd in pgc_catal:
        indx,   = np.where(pgc_catal==idd)
        R50u[i]  = R50u_catal[indx][0]
        R50g[i]  = R50g_catal[indx][0]
        R50r[i]  = R50r_catal[indx][0]
        R50i[i]  = R50i_catal[indx][0]
        R50z[i]  = R50z_catal[indx][0]
        R50w1[i] = R50w1_catal[indx][0]
        R50w2[i] = R50w2_catal[indx][0]
        Sba[i]   = Sba_catal[indx][0]
        Wba[i]   = Wba_catal[indx][0]


In [71]:
Nests = []
Cnames = []
PGCs  = [] 


for band in ['i', 'w1']:
    
    calib_Clusters = allCluster(table, band=band)

    for key in calib_Clusters:
        cluster = calib_Clusters[key]
        for i, idd in enumerate(cluster['pgc']):
            if not idd in PGCs:
                PGCs.append(idd)
                Nests.append(key)
                Cnames.append(cluster['name'])
        for i, idd in enumerate(cluster['pgc_w']):
            if not idd in PGCs:
                PGCs.append(idd)
                Nests.append(key)
                Cnames.append(cluster['name'])   

print '# of galaxies in Clusters: ', len(PGCs)            


n_zp = 0
for band in ['i', 'w1']:
    cluster = all_ZP(table, band=band)
    for i, idd in enumerate(cluster['pgc']):
        if not idd in PGCs:
            PGCs.append(idd)
            Nests.append('ZP')
            Cnames.append('ZP calibrator')
            n_zp += 1
    for i, idd in enumerate(cluster['pgc_w']):
        if not idd in PGCs:
            PGCs.append(idd)
            Nests.append('ZP')
            Cnames.append('ZP calibrator')               
            n_zp += 1
            


print '# of total galaxies: ', len(PGCs)
print '# of ZP galaxies: ', n_zp

# of galaxies in Clusters:  650
# of total galaxies:  743
# of ZP galaxies:  93


In [72]:

indx = np.argsort(PGCs)

Nrow = 0
for i in indx:
    
    row = ''
    idd = PGCs[i]
    
    j, = np.where(pgc == idd)
    j = j[0]
    
    if Cnames[i].strip()=='Virgo':
        Nrow+=1
        
        row += str(idd) + ' & '
        row += Name[j].strip()  + ' & '
        
        row += '%.0f'%inc[j] + '$\pm$' + '%.0f'%inc_e[j] + ' & '
        row += '%.0f'%Wmx[j] + '$\pm$' + '%.0f'%eWmx[j] + ' & '
        row += '%.3f'%logWimx[j] + '$\pm$' + '%.3f'%logWimx_e[j] + ' & '
        row += '%.3f'%m21[j] + '$\pm$' + '%.3f'%m21_e[j] + ' & '
        
        row += '%.2f'%uu[j] + ' & '
        row += '%.2f'%gg[j] + ' & '
        row += '%.2f'%rr[j] + ' & '
        row += '%.2f'%ii[j] + ' & '
        row += '%.2f'%zz[j] + ' & '
        row += '%.2f'%w1[j] + ' & '
        row += '%.2f'%w2[j] + ' & '
        
        row += '%.2f'%Sba[j] + ' & '
        row += '%.2f'%Wba[j] + ' & '

        row += '%.2f'%R50u[j] + ' & '
        row += '%.2f'%R50g[j] + ' & '
        row += '%.2f'%R50r[j] + ' & '
        row += '%.2f'%R50i[j] + ' & '
        row += '%.2f'%R50z[j] + ' & '
        row += '%.2f'%R50w1[j] + ' & '
        row += '%.2f'%R50w2[j] + ' \\\\ '        
        
        
        print row
        
    
    if Nrow==10:
        break
    

38803 & IC3033 & 59$\pm$5 & 111$\pm$1 & 2.112$\pm$0.023 & 15.811$\pm$0.076 & 15.89 & 14.93 & 14.55 & 14.34 & 14.31 & 14.91 & 15.21 & 0.46 & 0.53 & 0.36 & 0.35 & 0.33 & 0.33 & 0.31 & 0.38 & 0.49 \\ 
38943 & NGC4178 & 75$\pm$3 & 249$\pm$2 & 2.410$\pm$0.007 & 12.884$\pm$0.076 & 12.43 & 11.68 & 11.21 & 10.92 & 10.83 & 11.31 & 11.88 & 0.35 & 0.34 & 1.59 & 1.38 & 1.34 & 1.34 & 1.28 & 1.21 & 1.22 \\ 
38945 & IC3044 & 73$\pm$3 & 125$\pm$3 & 2.116$\pm$0.012 & 14.975$\pm$0.076 & 15.02 & 14.02 & 13.62 & 13.41 & 13.31 & 14.04 & 14.77 & 0.36 & 0.40 & 0.52 & 0.54 & 0.56 & 0.57 & 0.58 & 0.61 & 0.57 \\ 
39028 & NGC4192 & 78$\pm$3 & 458$\pm$2 & 2.671$\pm$0.005 & 12.724$\pm$0.076 & 11.57 & 10.49 & 9.77 & 9.36 & 9.13 & 9.38 & 9.98 & 0.24 & 0.36 & 2.76 & 1.96 & 1.82 & 1.77 & 1.71 & 1.39 & 1.40 \\ 
39181 & IC3066 & 83$\pm$3 & 132$\pm$8 & 2.123$\pm$0.025 & 16.958$\pm$0.076 & 16.70 & 15.38 & 14.84 & 14.57 & 14.44 & 15.14 & 15.79 & 0.33 & 0.33 & 0.22 & 0.25 & 0.25 & 0.26 & 0.26 & 0.29 & 0.29 \\ 
39224 & NGC42

In [74]:

indx = np.argsort(PGCs)

Nrow = 0
for i in indx:
    
    row = ''
    idd = PGCs[i]
    
    j, = np.where(pgc == idd)
    j = j[0]
    
    if Cnames[i].strip()=='Virgo':
        Nrow+=1
        
        row += str(idd) + ' & '
        row += Cnames[i].strip()  + ' & '
               
        row += '%.2f'%u_[j]  + ' & '
        row += '%.2f'%g_[j]  + ' & '
        row += '%.2f'%r_[j]  + ' & '
        row += '%.2f'%i_[j]  + ' & '
        row += '%.2f'%z_[j]  + ' & '
        row += '%.2f'%w1_[j] + ' & '
        row += '%.2f'%w2_[j] + ' & '
        
        
        row += '%.2f'%(u_[j]-u_s[j]) + '$\pm$' + '%.2f'%Au_e[j]  + ' & '
        row += '%.2f'%(g_[j]-g_s[j]) + '$\pm$' + '%.2f'%Ag_e[j]  + ' & '
        row += '%.2f'%(r_[j]-r_s[j]) + '$\pm$' + '%.2f'%Ar_e[j]  + ' & '
        row += '%.2f'%(i_[j]-i_s[j]) + '$\pm$' + '%.2f'%Ai_e[j]  + ' & '
        row += '%.2f'%(z_[j]-z_s[j]) + '$\pm$' + '%.2f'%Az_e[j]  + ' & '
        row += '%.2f'%(w1_[j]-w1_s[j]) + '$\pm$' + '%.2f'%Aw1_e[j] + ' & '
        row += '%.2f'%(w2_[j]-w2_s[j]) + '$\pm$' + '%.2f'%Aw2_e[j] + ' & '        
        
        
        row += '%.2f'%u_s[j]  + ' & '
        row += '%.2f'%g_s[j]  + ' & '
        row += '%.2f'%r_s[j]  + ' & '
        row += '%.2f'%i_s[j]  + ' & '
        row += '%.2f'%z_s[j]  + ' & '
        row += '%.2f'%w1_s[j] + ' & '
        row += '%.2f'%w2_s[j] + ' \\\\ '        
      
        
        
        print row
        
    
    if Nrow==10:
        break
    

38803 & Virgo & 15.75 & 14.83 & 14.48 & 14.29 & 14.27 & 14.91 & 15.21 & 0.24$\pm$0.05 & 0.16$\pm$0.03 & 0.11$\pm$0.02 & 0.09$\pm$0.02 & 0.07$\pm$0.02 & 0.01$\pm$0.00 & 0.00$\pm$0.00 & 15.51 & 14.67 & 14.37 & 14.20 & 14.20 & 14.90 & 15.21 \\ 
38943 & Virgo & 12.30 & 11.59 & 11.14 & 10.87 & 10.79 & 11.31 & 11.88 & 0.62$\pm$0.09 & 0.45$\pm$0.06 & 0.34$\pm$0.05 & 0.29$\pm$0.04 & 0.24$\pm$0.03 & 0.03$\pm$0.00 & 0.01$\pm$0.00 & 11.68 & 11.14 & 10.80 & 10.58 & 10.55 & 11.28 & 11.87 \\ 
38945 & Virgo & 14.87 & 13.90 & 13.54 & 13.35 & 13.26 & 14.03 & 14.76 & 0.43$\pm$0.06 & 0.27$\pm$0.04 & 0.19$\pm$0.03 & 0.16$\pm$0.02 & 0.12$\pm$0.02 & 0.00$\pm$0.00 & 0.00$\pm$0.00 & 14.44 & 13.63 & 13.35 & 13.19 & 13.14 & 14.03 & 14.76 \\ 
39028 & Virgo & 11.42 & 10.38 & 9.69 & 9.30 & 9.09 & 9.37 & 9.97 & 0.80$\pm$0.12 & 0.57$\pm$0.09 & 0.43$\pm$0.07 & 0.34$\pm$0.05 & 0.25$\pm$0.04 & 0.04$\pm$0.00 & 0.01$\pm$0.00 & 10.62 & 9.81 & 9.26 & 8.96 & 8.84 & 9.33 & 9.96 \\ 
39181 & Virgo & 16.56 & 15.28 & 14.77 & 14.